In [ ]:
import pandas as pd
import numpy as np
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error, make_scorer,r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor

### Chargement des données

In [13]:
train_data = pd.read_csv('data/x_train.csv')  
y_data = pd.read_csv('data/y_train.csv')
test_data = pd.read_csv('data/x_test.csv')

## Data Preparation

In [14]:
# Suppression de la colonne 'ID' (pas utile pour l'entraînement)
X = train_data.drop(columns=['ID'])
y = y_data.drop(columns=['ID'])

# Diviser les données en ensemble d'entraînement et de validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
def custom_weighted_rmse(y_true, y_pred):
    weights = np.where(y_true < 0.5, 1, 1.2)
    error_per_class = weights * (y_true - y_pred) ** 2
    mean_error = np.mean(error_per_class)
    return np.sqrt(mean_error)

custom_scorer = make_scorer(custom_weighted_rmse, greater_is_better=False)

## Etude classique 1 seul Model pour prédire 

### Selection des Modeles

On cherche à tester sur un grand nombre de datasets afin de voir si un modele marche mieux ou nom 

In [19]:
models = {
    'XGBoost' : XGBRegressor(),
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'ElasticNet': ElasticNet(),
    #'Support Vector Regression': SVR(),
    'Decision Tree Regression': DecisionTreeRegressor(),
    'Random Forest Regression': RandomForestRegressor(),
    #'Gradient Boosting Regression': GradientBoostingRegressor(),
    'K-Neighbors Regression': KNeighborsRegressor()}

res = list()
for mod in models.values() :
    print(f"Model: {mod.__class__.__name__}")
    # Initialiser le modèle
    model = mod
    # Entraîner le modèle de base
    model.fit(X_train, y_train)
    # Évaluation du modèle
    y_pred = model.predict(X_val)
    mse = mean_squared_error(y_val, y_pred)
    r2 = r2_score(y_val, y_pred) 
    custom_rmse = custom_weighted_rmse(y_val, y_pred)
    # Validation croisée
    #cv_scores = cross_val_score(model, X_train, y_train, cv=10, scoring=custom_scorer)
    res.append([mod.__class__.__name__,custom_rmse, mse,r2,])

Model: XGBRegressor
Model: LinearRegression
Model: Ridge
Model: Lasso


c:\Users\bapdu\COMMUN\Dauphine\Identification-Gaz-Toxiques-BERTIN\env\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 0.000e+00, tolerance: 0.000e+00
  model = cd_fast.enet_coordinate_descent(


Model: ElasticNet


c:\Users\bapdu\COMMUN\Dauphine\Identification-Gaz-Toxiques-BERTIN\env\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 0.000e+00, tolerance: 0.000e+00
  model = cd_fast.enet_coordinate_descent(


Model: DecisionTreeRegressor
Model: RandomForestRegressor
Model: KNeighborsRegressor


In [23]:
res = pd.DataFrame(res,columns=["model","custom_rmse","mse","r2"]).sort_values(by="custom_rmse",ascending=False)
# res["cv_mean"] = res['cv_score'].apply(lambda x : np.mean(x))
res

,model,custom_rmse,mse,r2
3,Lasso,0.217702,0.041588,0.043445
4,ElasticNet,0.217702,0.041588,0.043445
2,Ridge,0.178308,0.028143,0.272862
1,LinearRegression,0.178304,0.028143,0.272863
7,KNeighborsRegressor,0.026284,0.000649,0.973690
0,XGBRegressor,0.022887,0.000487,0.947358
5,DecisionTreeRegressor,0.014776,0.000204,0.993547
6,RandomForestRegressor,0.009530,0.000086,0.997001


Le meilleur model semble etre le random Forest Regressor. On va donc essayer d optimiser les parametres

### Optimisation des hyperparamètres - Random Forest

On va commencer par utiliser un modèle de Random Forest simple que l'on va evaluer

In [25]:
# model_rf = RandomForestRegressor(n_estimators=5, max_depth=7, min_samples_split=0.01, min_samples_leaf=30, random_state=42)
model_rf = RandomForestRegressor()
model_rf.fit(X_train, y_train)
y_pred = model_rf.predict(X_val)
rmse = custom_weighted_rmse(y_val, y_pred)
print(f"Erreur quadratique moyenne pondérée (RMSE) sur les données de validation : {rmse}")

Erreur quadratique moyenne pondérée (RMSE) sur les données de validation : 0.009469254954644969


Debut de l'optimisation des hyperparamètres

In [ ]:
# Optimisation des hyperparamètres avec GridSearchCV
param_grid = {
    'n_estimators': [5, 10, 20],
    'max_depth': [7, 10, 15],
    'min_samples_split': [0.01, 0.05, 0.1],
    'min_samples_leaf': [30, 50, 70]
}
# param_grid = {
#     'n_estimators': [11,12,13],
#     'max_depth': [15,20],
#     'min_samples_split': [0.01],
#     'min_samples_leaf': [30]
# }
grid_search = GridSearchCV(estimator=model_rf, param_grid=param_grid, cv=5, scoring=custom_scorer, verbose=2, n_jobs=-1)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

KeyboardInterrupt: 

In [ ]:
# Meilleurs hyperparamètres
print(f"Meilleurs hyperparamètres : {grid_search.best_params_}")

In [ ]:
# Évaluation finale sur les données de validation
y_pred = best_model.predict(X_val)
best_mse = mean_squared_error(y_val, y_pred)
best_r2 = r2_score(y_val, y_pred)
custom_mse = custom_weighted_rmse(y_val, y_pred)
print(f"Best Mean Squared Error: {best_mse}")
print(f"Best R^2 Score: {best_r2}")
print(f"Custom MSE sur les données de validation : {custom_mse}")

Erreur quadratique moyenne pondérée (RMSE) sur les données de validation : 0.09115404934287714


In [ ]:
# Effectuer une validation croisée à 5 folds
best_cv_scores = cross_val_score(best_model, X_train, y_train, cv=5, scoring=custom_scorer)
print(f"Best Cross-Validation Scores: {-best_cv_scores}")
print(f"Mean Best Cross-Validation Score : {-best_cv_scores.mean()}")

## Prediction de 1 model par Output

### Selection de modèles

In [ ]:
# Liste des modèles
models = {
    'XGBoost' : XGBRegressor(),
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'ElasticNet': ElasticNet(),
    'Support Vector Regression': SVR(),
    'Decision Tree Regression': DecisionTreeRegressor(),
    'Random Forest Regression': RandomForestRegressor(),
    'Gradient Boosting Regression': GradientBoostingRegressor(),
    'K-Neighbors Regression': KNeighborsRegressor()
}

res = list()

for mod in models.values():
    print(f"Model: {mod.__class__.__name__}")
    
    # Initialiser le modèle MultiOutputRegressor avec le modèle de base
    multi_target_model = MultiOutputRegressor(mod)
    
    # Entraîner le modèle multi-cible
    multi_target_model.fit(X_train, y_train)
    
    # Prédictions pour toutes les cibles
    y_pred = multi_target_model.predict(X_val)
    
    # Calcul des métriques pour chaque colonne cible
    mse = mean_squared_error(y_val, y_pred)
    r2 = r2_score(y_val, y_pred)
    custom_rmse = custom_weighted_rmse(y_val, y_pred)  # Remarque: tu peux ajuster cette fonction selon tes besoins
    
    # Ajouter les résultats dans la liste
    res.append([mod.__class__.__name__, custom_rmse, mse, r2])

In [ ]:
res = pd.DataFrame(res,columns=["model","custom_rmse","mse","r2"]).sort_values(by="custom_rmse",ascending=False)
# res["cv_mean"] = res['cv_score'].apply(lambda x : np.mean(x))
res.head(5)

## Test Pred

In [26]:
# 6. Prédictions finales (sur test_data, si disponible)
test_pred = model_rf.predict(test_data.drop(columns=['ID']))

In [27]:
# Enregistrement des résultats

output = pd.DataFrame(test_pred, columns=[f'c{i}' for i in range(1, 24)])
# Ajouter la colonne "ID" au début
output.insert(0, 'ID', test_data['ID'])
output.to_csv('predictions/predictions_simple_rf.csv', index=False)